In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise
import gurobipy as gp
from gurobipy import GRB
from demand_point import DemandPoint
from supply_point import SupplyPoint

demand_history_df = pd.read_csv('./dataset/Demand_History.csv')
limit = len(demand_history_df)
demand_points = []

for i in range(len(demand_history_df[:limit])):
    cur_row = demand_history_df.iloc[i]
    demand_object = DemandPoint(cur_row[0], cur_row[1], cur_row[2], np.array(cur_row[3:]))
    # demand_object.predict_demand()
    demand_points.append(demand_object)

existing_infra_df = pd.read_csv('./dataset/exisiting_EV_infrastructure_2018.csv')
supply_points = []

for i in range(len(existing_infra_df)):
    cur_row = existing_infra_df.iloc[i]
    supply_object = SupplyPoint(cur_row[0], cur_row[1], cur_row[2], cur_row[3], cur_row[4], cur_row[5])
    # supply_object.calculate_max_supply()
    supply_points.append(supply_object)

demand_point_locations = [[point.x, point.y] for point in demand_points]
supply_point_locations = [[point.x, point.y] for point in supply_points]
cost = pairwise.euclidean_distances(demand_point_locations, supply_point_locations)

model = gp.Model('Shell')

"""
Variables
----------

"""

allocation_matrix = {(demand_index, supply_index): model.addVar(vtype = GRB.CONTINUOUS,
                            lb = 0.0,
                            ub = GRB.INFINITY,
                            name = "x_{0}_{1}".format(demand_index, supply_index))
    for demand_index, demand_point in enumerate(demand_points) for supply_index, supply_point in enumerate(supply_points)}

num_fcs = {}
num_scs = {}

num_fcs = {(supply_index) : model.addVar(vtype = GRB.INTEGER,
                lb = supply_point.existing_num_FCS,
                ub = supply_point.total_parking_slots,
                name = "num_fcs_{0}".format(supply_index))
    for supply_index, supply_point in enumerate(supply_points)}

num_scs = {(supply_index) : model.addVar(vtype = GRB.INTEGER,
                lb = supply_point.existing_num_SCS,
                ub = supply_point.total_parking_slots,
                name = "num_scs_{0}".format(supply_index))
    for supply_index, supply_point in enumerate(supply_points)}

######TEST########
abs_demand_variation = {}

abs_demand_variation = {(demand_index) : model.addVar(vtype = GRB.CONTINUOUS,
                name = "abs_demand_variation_{0}".format(demand_index))
    for demand_index, demand_point in enumerate(demand_points)}

for demand_index, demand_point in enumerate(demand_points):
    z = model.addVar(lb=-GRB.INFINITY)
    model.addConstr(z == (demand_point.demand[0] - (gp.quicksum(allocation_matrix[demand_index, supply_index] for supply_index in range(len(supply_points))))))
    model.addGenConstrAbs(abs_demand_variation[demand_index], z, "absconstr_{0}".format(demand_index))
    
###################

"""
Constraints
------------

"""

model.addConstrs(
    (num_fcs[supply_index] + num_scs[supply_index] <= supply_point.total_parking_slots
    for supply_index, supply_point in enumerate(supply_points)),
    name = 'fcs_constr'
)

model.addConstrs(
    (gp.quicksum(allocation_matrix[demand_index, supply_index] 
    for demand_index in range(len(demand_points))) <= 400*num_fcs[supply_index] + 200*num_scs[supply_index] - 10e-3
    
    for supply_index, supply_point in enumerate(supply_points)),
    name = 'constr2'
)

# model.addConstrs(
#     (gp.quicksum(allocation_matrix[demand_index, supply_index] 
#     for supply_index in range(len(supply_points))) <= max(0, demand_point.demand[0] - 10e-3)

#     for demand_index, demand_point in enumerate(demand_points)),
#     name = 'constr3'
# )

In [ ]:
# sum = 0
# for demand_point in demand_points:
#     sum += demand_point.demand[0]
# print(sum)

In [ ]:
"""
Objective
"""

objective_terms = []
for demand_index, demand_point in enumerate(demand_points):
    for supply_index, supply_point in enumerate(supply_points):
        objective_terms.append(cost[demand_index][supply_index] * allocation_matrix[demand_index, supply_index])

# for demand_index, demand_point in enumerate(demand_points):
#     objective_terms.append(25 * (
#             demand_point.demand[0] - (gp.quicksum(allocation_matrix[demand_index, supply_index] for supply_index in range(len(supply_points))))
#         )
#     )
for demand_index, demand_point in enumerate(demand_points):
    objective_terms.append(24.5 * abs_demand_variation[demand_index])

for supply_index, supply_point in enumerate(supply_points):
    objective_terms.append(600 * (num_scs[supply_index] + 1.5 * num_fcs[supply_index]))

In [ ]:
model.update()
model.setObjective(gp.quicksum(objective_terms))
model.ModelSense = gp.GRB.MINIMIZE

In [ ]:
model.computeIIS()
model.write("model.ilp")

In [ ]:
model.feasRelaxS(0, True, False, True)
# model.feasRelaxS(0, True, True, True)


In [ ]:
model.setParam(GRB.Param.PoolSearchMode, 2)
model.setParam(GRB.Param.PoolSolutions, 3)

In [ ]:
model.optimize()

In [ ]:
model.setParam(GRB.Param.SolutionNumber, 2)

In [ ]:
%matplotlib qt 
import matplotlib.pyplot as plt

supply_points_x = []
supply_points_y = []
demand_points_x = []
demand_points_y = []

for demand_point in demand_points[::10]:
    for supply_point in supply_points:
        demand_index = int(demand_point.index)
        supply_index = int(supply_point.index)

        supply_points_x.append(supply_point.x)
        supply_points_y.append(supply_point.y)
        demand_points_x.append(demand_point.x)
        demand_points_y.append(demand_point.y)
        if allocation_matrix[demand_index, supply_index].X > 0:
            if(round(cost[demand_index][supply_index]) != round(math.dist([demand_point.x, demand_point.y], [supply_point.x, supply_point.y]))):
                print(demand_index, supply_index)
            # print(demand_index, supply_index, allocation_matrix[demand_index, supply_index].X)

            plt.text((supply_point.x + demand_point.x) / 2, (supply_point.y + demand_point.y) / 2, '%.1f'%(allocation_matrix[demand_index, supply_index].X), fontsize = 10)
            # plt.text(supply_point.x, supply_point.y , '%s'%(supply_index), fontsize = 10)
            # plt.text((supply_point.x + demand_point.x) / 2, (supply_point.y + demand_point.y) / 2, '%.1f'%(math.dist([demand_point.x, demand_point.y], [supply_point.x, supply_point.y])), fontsize = 10)
            # plt.text((supply_point.x + demand_point.x) / 2, (supply_point.y + demand_point.y) / 2, '%.1f'%(cost[demand_index][supply_index]), fontsize = 10)
            plt.plot([supply_point.x, demand_point.x], [supply_point.y, demand_point.y])

plt.scatter(supply_points_x, supply_points_y, color='r')
plt.scatter(demand_points_x, demand_points_y, color='b')
plt.show()


In [ ]:
allocation_copy = {}

for supply_index, supply_point in enumerate(supply_points):
    for demand_index in range(len(demand_points)): 
        allocation_copy[demand_index, supply_index] = allocation_matrix[demand_index, supply_index].X
        
for supply_index, supply_point in enumerate(supply_points):
    cur = sum([allocation_copy[demand_index, supply_index] for demand_index in range(len(demand_points))])
    max_allowed = supply_point.max_supply

    for demand_index in range(len(demand_points)): 
        if cur <= max_allowed:
            break
        
        cur -= allocation_copy[demand_index, supply_index]
        allocation_copy[demand_index, supply_index] = 0.0

In [ ]:
# Test constraint 1
for supply_index, supply_point in enumerate(supply_points):
    if num_fcs[supply_index].X < 0 or num_scs[supply_index].X < 0 or num_fcs[supply_index].X + num_scs[supply_index].X > supply_point.total_parking_slots:
        print(supply_index, num_fcs[supply_index].X, num_scs[supply_index].X)
    # print(supply_point.total_parking_slots - (num_fcs[supply_index].X + num_scs[supply_index].X))

In [ ]:
# Test constraint 2
for supply_index, supply_point in enumerate(supply_points):
    cur_sum = sum(allocation_matrix[demand_index, supply_index].X for demand_index in range(len(demand_points)))
    max_supply = 400 * num_fcs[supply_index].X + 200 * num_scs[supply_index].X
    if cur_sum > max_supply:
        print(supply_index, cur_sum, max_supply, cur_sum - max_supply)
    # if max_supply - cur_sum > 5:
    #     print(max_supply - cur_sum)

In [ ]:
# Test constraint 3
for demand_index, demand_point in enumerate(demand_points):
    cur_sum = sum(allocation_matrix[demand_index, supply_index].X for supply_index in range(len(supply_points)))
    max_sum = demand_point.demand[0]

    if abs(cur_sum - max_sum > 1):
        print(demand_index, cur_sum, max_sum, cur_sum - max_sum)
    # if max_sum - cur_sum > 0.5:
    #     print(demand_index, max_sum, cur_sum, max_sum - cur_sum)

In [ ]:
# Test constraint 4
for demand_index, demand_point in enumerate(demand_points):
    for supply_index, supply_point in enumerate(supply_points):
        if allocation_matrix[demand_index, supply_index].X < 0:
            print(demand_index, supply_index, allocation_matrix[demand_index, supply_index].X)

In [ ]:
for supply_index, supply_point in enumerate(supply_points):
    if supply_point.existing_num_FCS > num_fcs[supply_index].X:
        print(supply_index, supply_point.existing_num_FCS, num_fcs[supply_index].X)
    if supply_point.existing_num_SCS > num_scs[supply_index].X:
        print(supply_index, supply_point.existing_num_SCS, num_scs[supply_index].X)

In [ ]:
import csv  
header = ['year','data_type','demand_point_index','supply_point_index','value']
with open('shell_notebook.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write multiple rows
    for i in range(len(num_fcs)):
        writer.writerow(['2019', 'FCS', 0, i, int(math.ceil(num_fcs[i].X))])

In [ ]:
with open('shell_notebook.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    for i in range(len(num_scs)):
        writer.writerow(['2019', 'SCS', 0, i, int(math.ceil(num_scs[i].X))])

In [ ]:
with open('shell_notebook.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    for demand_index, demand_point in enumerate(demand_points):
        for supply_index, supply_point in enumerate(supply_points):
            writer.writerow(['2019', 'DS', demand_index, supply_index, max(0, allocation_matrix[demand_index, supply_index].X)])

In [ ]:
########################################################

In [ ]:
import csv  
header = ['year','data_type','demand_point_index','supply_point_index','value']
with open('shell_notebook.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write multiple rows
    for i in range(len(num_fcs)):
        writer.writerow(['2019', 'FCS', 0, i, int(math.ceil(num_fcs[i].Xn))])

In [ ]:
with open('shell_notebook.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    for i in range(len(num_scs)):
        writer.writerow(['2019', 'SCS', 0, i, int(math.ceil(num_scs[i].Xn))])

In [ ]:
with open('shell_notebook.csv', 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    for demand_index, demand_point in enumerate(demand_points):
        for supply_index, supply_point in enumerate(supply_points):
            writer.writerow(['2019', 'DS', demand_index, supply_index, max(0, allocation_matrix[demand_index, supply_index].Xn)])